## Imports

In [62]:
# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings
from intent_model import IntentModel

# Chat demo
import gradio as gr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data 

In [63]:
# Load the data from JSON file
with open("intent_recognition_v2.json", "r") as f:
    data = json.load(f)

train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])
    

# Intent recognition model

In [64]:
intent_model = IntentModel(train_data, label_map, labels)

In [65]:
import pandas as pd

# test the intent model
predicted_intent, confidence_scores = intent_model.get_intent("Can I give my subscription to someone else?")

# Retrieve the intent label and its corresponding confidence score
highest_confidence_intent = predicted_intent

print("Highest confidence intent:", highest_confidence_intent)
print("Highest confidence score:", confidence_scores)
print(len(confidence_scores))
print(len(intent_model.label_map.keys()))

pd.DataFrame({
    'keys': intent_model.label_map.keys(),
    'confidences': confidence_scores
})

Highest confidence intent: cancellation_legal_statement
Highest confidence score: [0.012756012380123138, 0.018988564610481262, 0.010949754156172276, 0.008784662932157516, 0.01938551664352417, 0.04217369854450226, 0.04235825315117836, 0.023837264627218246, 0.04633849859237671, 0.021925723180174828, 0.021904442459344864, 0.08517713099718094, 0.026409432291984558, 0.0069831800647079945, 0.0267922505736351, 0.2928673326969147, 0.026343459263443947, 0.016675269231200218, 0.009689160622656345, 0.209237739443779, 0.007365390658378601, 0.023057281970977783]
22
22


,keys,confidences
0,third_parties,0.012756
1,legal_statement_information,0.018989
2,minimum_age,0.010950
3,personal_information,0.008785
4,use_restrictions_legal_statement,0.019386
5,privacy_policy_updates,0.042174
6,delete_personal_information,0.042358
7,access_personal_data,0.023837
8,data_breaches,0.046338
9,data_collection,0.021926


## ChatGPT model

In [66]:
# GPT model here
gpt_model = GPT()
privacy_level = 2 # set to 2 for maximum privacy
minimum_confidence_score = 0.5

# gpt_model = False

# Test ChatGPT model
# gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


d:\school\iYYU\gpt.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  serie = serie.str.replace('\\n', ' ')


# Load Settings class

In [67]:
settings_model = Settings()

# Test the settings model
# settings_model.update_user_setting("1", "profile_published", False)
settings_model.set_user_calling_card_visibility(user_id = "0", individual_user_id = "2", profile_card_component = "profile_card_component_3_visible", component_state = True)



## Get response using intent

In [68]:
def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

# Set up gradio chat for demo

In [75]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False,
             tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent, confidence_score = intent_model.get_intent(question=user_message)
        print('intent:', intent)
        print('confidence:', confidence_score)

        confidence_score = max(confidence_score)

        intent = get_object_by_intent(intent)


        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent['use_gpt']
        generate_context = True

        if confidence_score >= 0.5:
            # model is confident on the intent
            question_to_ask = user_message if privacy_level <= 1 or not use_gpt or len(intent['responses']) == 0 else intent['responses'][0]
        else:
            use_gpt = True
            generate_context = False
            question_to_ask = user_message if privacy_level <= 1 else 'Say that you are not sure about the question that the user asked, that they maybe have to rephrase.'

        # if the intent is 'privacy_policy' or 'legal_statement', use GPT to generate a response
        response = ('<em><strong>(ChatGPT - intent found ({}): {}):</strong></em> \n{}'.format(confidence_score, intent['intent'], gpt_model.answer_question(question=question_to_ask, with_context=generate_context))
                    if use_gpt else
                    '<em><strong>(intent found ({}): {}):</strong></em> \n{}'.format(confidence_score, intent['intent'], intent['responses'][0]))
        # response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message)) if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])

        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        if not use_gpt:
            time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.launch()


Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


intent: use_restrictions_legal_statement
confidence: [0.0416993647813797, 0.07000138610601425, 0.059815358370542526, 0.05363086983561516, 0.09074132889509201, 0.0405288003385067, 0.019422655925154686, 0.023672662675380707, 0.04441330209374428, 0.0684223547577858, 0.04165655001997948, 0.036616433411836624, 0.02668195776641369, 0.02987222746014595, 0.011195588856935501, 0.04795532301068306, 0.08399158716201782, 0.05283616855740547, 0.038445230573415756, 0.035350922495126724, 0.0741099938750267, 0.008940017782151699]
